<a href="https://colab.research.google.com/github/martinpdab/asapa/blob/main/machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sklearn

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import  train_test_split
from keras.utils.np_utils import to_categorical

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
URL = 'https://raw.githubusercontent.com/martinpdab/asapa/main/dataset/dataset-kasus-shuffled%20(1).csv'
df = pd.read_csv(URL)
df.head()

,tempat,jenis,banyak_kekerasan,banyak_korban,usia,hubungan_pelaku,label
0,rumah-tangga,seksual,1,1,remaja,orang-tua,2
1,fasilitas-umum,seksual,1,1,dewasa,lainnya,1
2,rumah-tangga,penelantaran,1,1,balita,keluarga/saudara,2
3,rumah-tangga,psikis,1,1,remaja,orang-tua,0
4,rumah-tangga,fisik,1,1,remaja,orang-tua,2


In [33]:
df.isnull().any()

x = df.drop("label", 1)
y = df["label"]


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Y_train_binary = y_train.copy()
Y_test_binary = y_test.copy()

Y_train_binary[Y_train_binary > 0] = 1
Y_test_binary[Y_test_binary > 0] = 1

x_array = np.asarray(x_train)
y_array = np.asarray(y_train)
X_array = np.asarray(x_test)
Y_array = np.asarray(y_test)


print(X_array)

[['lainnya' 'lainnya' 1 1 'dewasa' 'lainnya']
 ['lainnya' 'psikis' 1 1 'dewasa' 'pacar/teman']
 ['lainnya' 'fisik' 2 1 'balita' 'orang-tua']
 ...
 ['rumah-tangga' 'seksual' 1 1 'dewasa' 'suami/istri']
 ['fasilitas-umum' 'fisik' 1 1 'dewasa' 'suami/istri']
 ['lainnya' 'seksual' 1 1 'dewasa' 'lainnya']]


In [68]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [69]:
for feature_batch, label_batch in train_ds.take(1):
  print('Parameter:', list(feature_batch.keys()))
  print('Tempat Kejadian:', feature_batch['tempat'])
  print('Jenis Kekerasan:', feature_batch['jenis'])
  print('Banyak Kekerasan:', feature_batch['banyak_kekerasan'])
  print('Banyak Korban:', feature_batch['banyak_korban'])
  print('Usia Korban:', feature_batch['usia_korban'])
  print('Hubungan Pelaku dengan Korban:', feature_batch['hubungan_pelaku'])
  print('Label:', label_batch )

Parameter: ['tempat', 'jenis', 'banyak_kekerasan', 'banyak_korban', 'usia_korban', 'hubungan_pelaku']
Tempat Kejadian: tf.Tensor(
[b'fasilitas-umum' b'rumah-tangga' b'lainnya' b'rumah-tangga'
 b'rumah-tangga'], shape=(5,), dtype=string)
Jenis Kekerasan: tf.Tensor([b'psikis' b'psikis' b'fisik' b'psikis' b'fisik'], shape=(5,), dtype=string)
Banyak Kekerasan: tf.Tensor([1 1 2 1 1], shape=(5,), dtype=int64)
Banyak Korban: tf.Tensor([1 1 1 1 1], shape=(5,), dtype=int64)
Usia Korban: tf.Tensor([b'dewasa' b'dewasa' b'dewasa' b'balita' b'remaja'], shape=(5,), dtype=string)
Hubungan Pelaku dengan Korban: tf.Tensor([b'lainnya' b'suami/istri' b'lainnya' b'keluarga/saudara' b'orang-tua'], shape=(5,), dtype=string)
Label: tf.Tensor([1 1 3 2 3], shape=(5,), dtype=int64)


In [7]:
feature_columns = []

feature_columns.append(feature_column.numeric_column('banyak_korban'))
banyak_kekerasan = feature_column.numeric_column('banyak_kekerasan')
feature_columns.append(banyak_kekerasan)

tempat_kejadian = tf.feature_column.categorical_column_with_vocabulary_list(
                'tempat', ['rumah-tangga', 'tempat-kerja', 'sekolah', 'fasilitas-umum', 'lainnya'])
tempat_one_hot = tf.feature_column.indicator_column(tempat_kejadian)
feature_columns.append(tempat_one_hot)

jenis_kekerasan = tf.feature_column.categorical_column_with_vocabulary_list(
    'jenis', ['fisik', 'seksual', 'psikis', 'penelantaran', 'lainnya', 'eksploitasi', 'traficking'])
jenis_one_hot = tf.feature_column.indicator_column(jenis_kekerasan)
feature_columns.append(jenis_one_hot)

usia_korban = tf.feature_column.categorical_column_with_vocabulary_list(
    'usia_korban', ['balita', 'anak-anak', 'remaja', 'dewasa', 'lansia'])
usia_one_hot = tf.feature_column.indicator_column(usia_korban)
feature_columns.append(usia_one_hot)

hubungan_pelaku = tf.feature_column.categorical_column_with_vocabulary_list(
    'hubungan_pelaku', ['orang-tua', 'keluarga/saudara', 'tetangga', 'suami/istri', 'majikan', 'rekan-kerja', 'guru', 'pacar/teman', 'lainnya'])
hubungan_one_hot = tf.feature_column.indicator_column(hubungan_pelaku)
feature_columns.append(hubungan_one_hot)



feature_columns

[NumericColumn(key='banyak_korban', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='banyak_kekerasan', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='tempat', vocabulary_list=('rumah-tangga', 'tempat-kerja', 'sekolah', 'fasilitas-umum', 'lainnya'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='jenis', vocabulary_list=('fisik', 'seksual', 'psikis', 'penelantaran', 'lainnya', 'eksploitasi', 'traficking'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='usia_korban', vocabulary_list=('balita', 'anak-anak', 'remaja', 'dewasa', 'lansia'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='hubungan_pelaku', vocabular

In [43]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer

In [9]:
print(df['label'])

0       2
1       1
2       2
3       0
4       2
       ..
5214    2
5215    0
5216    2
5217    0
5218    2
Name: label, Length: 5219, dtype: int64


In [24]:
batch_size = 32
train_ds = df_to_dataset(y_train, batch_size=batch_size)
test_ds = df_to_dataset(y_train, batch_size=batch_size)

AttributeError: ignored

In [73]:
estimator = tf.estimator.DNNClassifier(hidden_units=[256, 128, 64], 
                                       feature_columns=feature_columns, 
                                       n_classes=3, 
                                       model_dir='graphs/dnn')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'graphs/dnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [42]:
feature_layer.shape()

AttributeError: ignored

In [41]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [38]:
model.summary()

ValueError: ignored